# StateMachineIterators

This package implements a general-purpose finite state machine, consisting of a set of states and transitions between those states. Each transition is defined by a function, which returns `true` when that transition should occur. 

It also defines the StateMachineIterator type, which allows finite state machines to be used as generic Julia iterators. 

For example, here is a simple state machine, inspired by the example from [FiniteStateMachine.jl](https://github.com/tinybike/FiniteStateMachine.jl):


In [1]:
using StateMachineIterators

In [2]:
# Construct a new state machine. The syntax is:
# StateMachine(initial_state, final_state, transitions)
# where each transition is a tuple of:
#   (from_state, check_function, to_state)
machine = StateMachine(:first, :fourth, 
    [(:first, event -> event == :hop, :second),
     (:second, event -> event == :skip, :third),
     (:third, event -> event == :jump, :fourth)])

StateMachine with states: [:third,:first,:second,:fourth]
    initial state: first
    final state: fourth

We can step through the state machine by calling nextstate(). Note that this returns a new state, rather than mutating the state machine itself.

In [3]:
state = initial(machine)

:first

In [4]:
state = nextstate(machine, state, :hop)

:second

In [5]:
state = nextstate(machine, state, :skip)

:third

In [6]:
state = nextstate(machine, state, :jump)

:fourth

In [7]:
finished(machine, state)

true

Let's step through all the states in the state machine:

In [8]:
state = initial(machine)
@show state
while !finished(machine, state)
    if state == :first
        event = :hop
    elseif state == :second
        event = :skip
    else
        event = :jump
    end
    state = nextstate(machine, state, event)
    @show state
end

state = :first
state = :second
state = :third
state = :fourth


This should look suspiciously like the generic Julia iterator protocol. To use the state machine as a real iterator, we just need to provide it with a function returning the input to its transition functions:

In [9]:
event = :hop
for state in StateMachineIterator(machine, () -> event)
    if state == :first
        event = :hop
    elseif state == :second
        event = :skip
    else
        event = :jump
    end
    @show state
end
@show state

state = :second
state = :third
state = :fourth
state = :fourth


:fourth

Let's look at a more complex example. We'll use a state machine to define a simple oscillator to increment our variable x until x >= 10, then decrement it until x <= 1, then terminate. 

In [10]:
machine = StateMachine(:going_up, :final, 
   [(:going_up, x -> x >= 10, :going_down),
    (:going_down, x -> x <= 1, :final)])

behaviors = Dict(:going_up => x -> x + 1,
                 :going_down => x -> x - 1,
                 :final => x -> x)

# Iterate through states of the machine
x = 1
state = initial(machine)
while !finished(machine, state)
    state = nextstate(machine, state, x)
    println("x: ", x, "\tstate: ", state)
    x = behaviors[state](x)
end

x: 1	state: going_up
x: 2	state: going_up
x: 3	state: going_up
x: 4	state: going_up
x: 5	state: going_up
x: 6	state: going_up
x: 7	state: going_up
x: 8	state: going_up
x: 9	state: going_up
x: 10	state: going_down
x: 9	state: going_down
x: 8	state: going_down
x: 7	state: going_down
x: 6	state: going_down
x: 5	state: going_down
x: 4	state: going_down
x: 3	state: going_down
x: 2	state: going_down
x: 1	state: final


Using the StateMachineIterator makes this a bit simpler:

In [11]:
machine = StateMachine(:going_up, :final, 
   [(:going_up, x -> x >= 10, :going_down),
    (:going_down, x -> x <= 1, :final)])

behaviors = Dict(:going_up => x -> x + 1,
                 :going_down => x -> x - 1,
                 :final => x -> x)

x = 1
for state in StateMachineIterator(machine, () -> x)
    println("x: ", x, "\tstate: ", state)
    x = behaviors[state](x)
end

x: 1	state: going_up
x: 2	state: going_up
x: 3	state: going_up
x: 4	state: going_up
x: 5	state: going_up
x: 6	state: going_up
x: 7	state: going_up
x: 8	state: going_up
x: 9	state: going_up
x: 10	state: going_down
x: 9	state: going_down
x: 8	state: going_down
x: 7	state: going_down
x: 6	state: going_down
x: 5	state: going_down
x: 4	state: going_down
x: 3	state: going_down
x: 2	state: going_down
x: 1	state: final


Now that we have an iterator, we can use the rest of Julia's general iterator routines to combine state machines. Here's a simple example, in which x rises from 1 to 10 then falls back to 1, while y oscillates between 1 and 3 repeatedly:

In [12]:
x_machine = StateMachine(:going_up, :final, 
   [(:going_up, x -> x >= 10, :going_down),
    (:going_down, x -> x <= 1, :final)]) # terminate when x gets back down to 1

y_machine = StateMachine(:going_up, :final,
   [(:going_up, y -> y >= 3, :going_down),
    (:going_down, y -> y <= 1, :going_up)]) # this state machine never terminates

behaviors = Dict(:going_up => a -> a + 1,
                 :going_down => a -> a - 1,
                 :final => a -> a)

x = 1
y = 1
# Julia's Base.zip(iter1, iter2) lets us simultaneously execute two 
# state machines until either of them terminates.
for (x_state, y_state) in zip(StateMachineIterator(x_machine, () -> x),
                              StateMachineIterator(y_machine, () -> y))
    println("x: ", x, "\ty: ", y)
    x = behaviors[x_state](x)
    y = behaviors[y_state](y)
end

x: 1	y: 1
x: 2	y: 2
x: 3	y: 3
x: 4	y: 2
x: 5	y: 1
x: 6	y: 2
x: 7	y: 3
x: 8	y: 2
x: 9	y: 1
x: 10	y: 2
x: 9	y: 3
x: 8	y: 2
x: 7	y: 1
x: 6	y: 2
x: 5	y: 3
x: 4	y: 2
x: 3	y: 1
x: 2	y: 2
x: 1	y: 3


We can also execute a state machine repeatedly with more of Julia's iterator tools:

In [13]:
import Iterators: chain

x = 1
for state in chain(repeated(StateMachineIterator(machine, () -> x), 2)...)
    println("x: ", x, "\tstate: ", state)
    x = behaviors[state](x)
end

x: 1	state: going_up
x: 2	state: going_up
x: 3	state: going_up
x: 4	state: going_up
x: 5	state: going_up
x: 6	state: going_up
x: 7	state: going_up
x: 8	state: going_up
x: 9	state: going_up
x: 10	state: going_down
x: 9	state: going_down
x: 8	state: going_down
x: 7	state: going_down
x: 6	state: going_down
x: 5	state: going_down
x: 4	state: going_down
x: 3	state: going_down
x: 2	state: going_down
x: 1	state: final
x: 1	state: going_up
x: 2	state: going_up
x: 3	state: going_up
x: 4	state: going_up
x: 5	state: going_up
x: 6	state: going_up
x: 7	state: going_up
x: 8	state: going_up
x: 9	state: going_up
x: 10	state: going_down
x: 9	state: going_down
x: 8	state: going_down
x: 7	state: going_down
x: 6	state: going_down
x: 5	state: going_down
x: 4	state: going_down
x: 3	state: going_down
x: 2	state: going_down
x: 1	state: final
